## Full Keras with augmentation

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import skimage 
from scipy import ndimage, misc

In [3]:
import cv2
def plot_image(file_path):
    """ Print image from file 'file_path' """
    img = cv2.imread(file_path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.show()

In [49]:
from keras.utils import np_utils
from sklearn.datasets import load_files

data = load_files('prep13/train')
caltech_files_train = np.array(data['filenames'])
caltech_targets_train = np.array(data['target'])

data_val = load_files('prep13/val')
caltech_files_val = np.array(data_val['filenames'])
caltech_targets_val = np.array(data_val['target'])

X_train, X_val = caltech_files_train, caltech_files_val
y_train_class, y_val_class = caltech_targets_train, caltech_targets_val
y_train = np_utils.to_categorical(y_train_class, 256)
y_val = np_utils.to_categorical(y_val_class, 256)

del data, data_val, caltech_files_train, caltech_targets_train, caltech_files_val, caltech_targets_val

In [50]:
test_files = ['prep13/test/'+ file for file in os.listdir('prep13/test')]
#test_files[:5]

In [52]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

resize = (64, 64)

CHANNELS = 3
ROWS = resize[0]
COLS = resize[1]
mean = 135

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')

## Data load and resize

In [53]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path, resize = resize):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=resize)
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths, resize = resize):
    list_of_tensors = [path_to_tensor(img_path, resize) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [54]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_data = paths_to_tensor(X_train, resize)/255

100%|██████████| 18772/18772 [00:04<00:00, 3963.19it/s]


In [55]:
valid_data = paths_to_tensor(X_val, resize)/255

100%|██████████| 3328/3328 [00:02<00:00, 1402.53it/s]


In [56]:
test_data = paths_to_tensor(test_files, resize)/255

100%|██████████| 7680/7680 [00:01<00:00, 4292.49it/s]


In [57]:
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

(18772, 64, 64, 3)
(3328, 64, 64, 3)
(7680, 64, 64, 3)


In [58]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        shear_range=0.1,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

# only rescaling for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'prep13/train',  # this is the target directory
        target_size=(ROWS, COLS),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = train_datagen.flow_from_directory(
        'prep13/val',
        target_size=(ROWS, COLS),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    'prep13/test',
    target_size=(64, 64),
    class_mode=None,
    batch_size=64,
    shuffle=False)

Found 18772 images belonging to 256 classes.
Found 3328 images belonging to 256 classes.
Found 0 images belonging to 0 classes.


In [59]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', 
                        input_shape=(resize[0], resize[1], CHANNELS)))
model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(scale = False))
model.add(Dropout(0.4))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(scale = False))
model.add(Dropout(0.4))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(scale = False))
model.add(Dropout(0.4))
model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(scale = False))
model.add(Dropout(0.4))
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 64, 64, 16)        448       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 64, 64, 16)        2320      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 16)        48        
_________________________________________________________________
dropout_13 (Dropout)         (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 32, 32, 32)        4640      
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 32, 32, 32)        9248      
__________

In [60]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [61]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

epochs = 30

checkpointer = [ModelCheckpoint(filepath='saved_models/unival13_new.hdf5', 
                               verbose=1, save_best_only=True),
               ReduceLROnPlateau(verbose=1, patience=3, factor = 0.1), 
               EarlyStopping(verbose=1, patience=5, monitor='val_loss')]

#model.fit(train, y_train, 
#          validation_data=(val, y_val),
#          epochs=epochs, batch_size=64, callbacks=checkpointer, verbose=1)

In [62]:
model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_data) // batch_size +1,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(valid_data) // batch_size+1, callbacks=checkpointer, verbose=1)

Epoch 1/30
1174/1174 [==============================] - 24s - loss: 4.9535 - acc: 0.0969 - val_loss: 5.5956 - val_acc: 0.0329
Epoch 2/30
1174/1174 [==============================] - 24s - loss: 4.5299 - acc: 0.1378 - val_loss: 5.1774 - val_acc: 0.0574
Epoch 3/30
1174/1174 [==============================] - 24s - loss: 4.2708 - acc: 0.1627 - val_loss: 5.0064 - val_acc: 0.0682
Epoch 4/30
1174/1174 [==============================] - 24s - loss: 4.0744 - acc: 0.1892 - val_loss: 4.8829 - val_acc: 0.0921
Epoch 5/30
1174/1174 [==============================] - 23s - loss: 3.9060 - acc: 0.2067 - val_loss: 4.8872 - val_acc: 0.0870
Epoch 6/30
1174/1174 [==============================] - 24s - loss: 3.7678 - acc: 0.2257 - val_loss: 4.4950 - val_acc: 0.1175
Epoch 7/30
1174/1174 [==============================] - 24s - loss: 3.6551 - acc: 0.2399 - val_loss: 4.3040 - val_acc: 0.1408
Epoch 8/30
1174/1174 [==============================] - 24s - loss: 3.5353 - acc: 0.2548 - val_loss: 4.4492 - val_acc:

In [63]:
model.load_weights('saved_models/unival13_new.hdf5')
predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in valid_data]
# get val accuracy
val_accuracy = 100*np.sum(np.array(predictions)==np.argmax(y_val, axis=1))/len(predictions)
print('Validation accuracy: %.2f%%' % val_accuracy)

Validation accuracy: 22.06%


In [64]:
predictions_test = [1+np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_data]

In [65]:
image = [item.split('/')[2] for item in test_files]
submission = pd.DataFrame(
    {'image': image,
     'class': predictions_test
    })
submission = submission[['image','class']]
submission.head()

,image,class
0,16800.jpg,148
1,19320.jpg,161
2,21505.jpg,203
3,11014.jpg,21
4,10790.jpg,253


In [66]:
submission.to_csv('submissions/keras_aug_unival13.csv', index = False)

In [39]:
#keras_aug1.hdf5 - 28.17%
#keras_aug2.hdf5 - 29.08% - 3 conv layers
#normval64.hdf5 - 38.34%
#unival64.hdf5 - 13.77%
#normval64_new.hdf5 - 35.70%
#unival_13.hdf5 - 22.06%